In [1]:
import json
import copy
from collections import OrderedDict

In [2]:
json_path = './data/knowledge_tags.json'
with open(json_path, 'r', encoding='utf-8') as j :
    knowledge_tags = json.load(j)

In [3]:
# concept을 리스트로 추출 + 중복 제거
concepts = []
concept_ids = []
for knowledge_tag in knowledge_tags.values() :
    if knowledge_tag['fromConcept']['id'] in concept_ids :
        continue
    concepts.append(knowledge_tag['fromConcept'])
    concept_ids.append(knowledge_tag['fromConcept']['id'])
for knowledge_tag in knowledge_tags.values() :
    if knowledge_tag['toConcept']['id'] in concept_ids :
        continue
    concepts.append(knowledge_tag['toConcept'])
    concept_ids.append(knowledge_tag['toConcept']['id'])

In [4]:
# 1631개
len(concept_ids)

1631

In [5]:
# concept_id_list.json으로 저장
with open('./data/json/concept_id_list.json', 'w', encoding='utf-8') as f:
    json.dump(concept_ids, f, ensure_ascii=False)

In [6]:
# 퀵정렬에서 비교하는 기준을 1)chapter_id 2)acheivement_id로 설정
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left, right, equal = [], [], []
    for i in arr:
        if int(i['chapter']['id']) < int(pivot['chapter']['id']):
            left.append(i)
        elif int(i['chapter']['id']) > int(pivot['chapter']['id']):
            right.append(i)
        else:
            equal.append(i)
    # chapter_id가 같은 애들끼리 achievement_id 가지고 다시 정렬
    quick_sort_ach(equal)
    return quick_sort(left) + equal + quick_sort(right)

def quick_sort_ach(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left, right, equal = [], [], []
    for i in arr:
        if int(i['achievement']['id']) < int(pivot['achievement']['id']):
            left.append(i)
        elif int(i['achievement']['id']) > int(pivot['achievement']['id']):
            right.append(i)
        else:
            equal.append(i)
    return quick_sort(left) + equal + quick_sort(right)

In [7]:
# 정렬된 concepts 리스트
concepts_sorted = quick_sort(concepts)

In [8]:
# knowledgeTag_skillID.txt 파일에서 skill_dic으로
# (키) concept_id : (밸류) skill_id

txt_path = './data/knowledgeTag_skillID.txt'
skill_dic = {}
with open(txt_path, "r") as file:
    lines = file.readlines()
    for line in lines:
        data = line.strip().split('\t')
        skill_dic[int(data[0])] = int(data[1])

### ㅇ concept에서 chapter 삭제 안 한 버전

In [9]:
# 필요한 부분 세분화 시키고, skill_id 추가

concepts_neo4j = copy.deepcopy(concepts_sorted) # 내부에 딕셔너리가 있는 리스트는 .copy()나 슬라이싱[:]으로는 부족해

for concept in concepts_neo4j :
    # semester 한 번 이름 변경 후 사용하기 (순서 맞추기용)
    concept['new_key'] = concept.pop('semester')
    temp = concept['new_key'].split('-')
    concept['school_level'] = temp[0].strip()
    concept['grade_level'] = temp[1].strip()
    concept['semester'] = temp[2].strip()
    del concept['new_key']

    concept['chapter_id'] = int(concept['chapter']['id'])
    temp2 = concept['chapter']['name'].split('>')
    if len(temp2)==3:
        concept['chapter_name']=temp2[2].strip()
        concept['chapter_main']=temp2[0].strip()
        concept['chapter_sub']=temp2[1].strip()
    # 초등은 대단원 없음   
    else:
        concept['chapter_name']=temp2[1].strip()
        concept['chapter_main']=""
        concept['chapter_sub']=temp2[0].strip()
    concept['achievement_id'] = int(concept['achievement']['id'])
    concept['achievement_name'] = concept['achievement']['name']
    del concept['chapter']
    del concept['achievement']
    concept['skill_id'] = skill_dic[concept['id']]

In [10]:
# concepts_neo4j.json으로 저장
with open('./data/json/concepts_neo4j.json', 'w', encoding='utf-8') as f:
    json.dump(concepts_neo4j, f, ensure_ascii=False)
concepts_neo4j[0]

{'id': 5814,
 'name': '1부터 5까지의 수',
 'description': '1부터 5까지의 수를 읽고 셀 수 있다.',
 'school_level': '초등',
 'grade_level': '초1',
 'semester': '1학기',
 'chapter_id': 1,
 'chapter_name': '몇일까요 (1)',
 'chapter_main': '',
 'chapter_sub': '9까지의 수',
 'achievement_id': 1,
 'achievement_name': '50까지의수 개념을이해 하고, 수를 세고 읽고 쓸 수 있다.',
 'skill_id': 8}

In [11]:
concepts_sorted[0]

{'id': 5814,
 'name': '1부터 5까지의 수',
 'semester': '초등-초1-1학기',
 'description': '1부터 5까지의 수를 읽고 셀 수 있다.',
 'chapter': {'id': '1', 'name': '9까지의 수 > 몇일까요 (1)'},
 'achievement': {'id': '1', 'name': '50까지의수 개념을이해 하고, 수를 세고 읽고 쓸 수 있다.'}}

### ㅇ concept에서 chapter 추출 버전

In [12]:
# concepts_sort에서 
# 1 : chapter 관련 부분 : 세분화 -> 추출 -> chapters에 담기
# 2 : 필요없는 부분 삭제
# 3 : skill_id 추가
chapters = []

for concept in concepts_sorted :
    chapter = {}
    chapter['chapter_id'] = int(concept['chapter']['id'])
    concept['concept_chapter_id'] = int(concept['chapter']['id'])
    temp2 = concept['chapter']['name'].split('>')
    if len(temp2)==3:
        chapter['chapter_name']=temp2[2].strip()
    else:
        chapter['chapter_name']=temp2[1].strip()
    temp = concept['semester'].split('-')
    chapter['school_level'] = temp[0].strip()
    chapter['grade_level'] = temp[1].strip()
    chapter['semester'] = temp[2].strip()
    del concept['semester'] 
    if len(temp2)==3:
        chapter['chapter_main']=temp2[0].strip()
        chapter['chapter_sub']=temp2[1].strip()
    # 초등은 대단원 없음   
    else:
        chapter['chapter_main']=""
        chapter['chapter_sub']=temp2[0].strip()
    concept['achievement_id'] = int(concept['achievement']['id'])
    concept['achievement_name'] = concept['achievement']['name']
    del concept['chapter']
    del concept['achievement']
    concept['skill_id'] = skill_dic[concept['id']]
    chapters.append(chapter)

In [13]:
concepts_sorted[0]

{'id': 5814,
 'name': '1부터 5까지의 수',
 'description': '1부터 5까지의 수를 읽고 셀 수 있다.',
 'concept_chapter_id': 1,
 'achievement_id': 1,
 'achievement_name': '50까지의수 개념을이해 하고, 수를 세고 읽고 쓸 수 있다.',
 'skill_id': 8}

In [14]:
chapters[0]

{'chapter_id': 1,
 'chapter_name': '몇일까요 (1)',
 'school_level': '초등',
 'grade_level': '초1',
 'semester': '1학기',
 'chapter_main': '',
 'chapter_sub': '9까지의 수'}

In [15]:
# chapters 중복 제거
chapters_unique = [dict(t) for t in {tuple(chapter.items()) for chapter in chapters}]

In [16]:
len(chapters_unique)

647

In [17]:
chapter_ids = []
for chapter in chapters:
    chapter_ids.append(chapter['chapter_id'])
# chapter_ids_unigue = list(set(chapter_ids))
# 순서 유지하면서 중복제거하기
chapter_ids_unigue = list(OrderedDict.fromkeys(chapter_ids))
len(chapter_ids_unigue)

647

In [18]:
### 중복되는 데이터 확인해서 원본 파일 수정함

In [19]:
# concept_id는 같은데 내용은 다른 데이터가 있나봐. 확인해보자
duple_list = []
for chapter01 in chapters_unique:
    check = []
    for chapter02 in chapters_unique:
        if chapter01['chapter_id']==chapter02['chapter_id']:
            check.append(chapter01)
            check.append(chapter02)
    if check !=[] :
        duple_list.append(check)

In [20]:
for i, duple in enumerate(duple_list):
    if duple[0]!=duple[1]:
        print(i)

In [21]:
# 확인해보니 띄어쓰기의 문제야
# 각각 확인해서 원본 파일 수정함
# duple_list[654]

In [22]:
# 그러면 그냥 유니크 아이디 돌려서 처음 만나는 데이터만 추출해도 돼!!
chapters_unique2 = []
for chapter_id in chapter_ids_unigue:
    for chapter in chapters:
        if chapter['chapter_id']==chapter_id:
            chapters_unique2.append(chapter)
            break
len(chapters_unique2)

647

In [23]:
chapters_unique2[0]

{'chapter_id': 1,
 'chapter_name': '몇일까요 (1)',
 'school_level': '초등',
 'grade_level': '초1',
 'semester': '1학기',
 'chapter_main': '',
 'chapter_sub': '9까지의 수'}

### ㅇ concepts_list.json으로 저장

In [24]:
# concepts_list.json으로 저장
with open('./data/json/concepts_list.json', 'w', encoding='utf-8') as f:
    json.dump(concepts_sorted, f, ensure_ascii=False)
concepts_sorted[0]

{'id': 5814,
 'name': '1부터 5까지의 수',
 'description': '1부터 5까지의 수를 읽고 셀 수 있다.',
 'concept_chapter_id': 1,
 'achievement_id': 1,
 'achievement_name': '50까지의수 개념을이해 하고, 수를 세고 읽고 쓸 수 있다.',
 'skill_id': 8}

### ㅇ chapters_list.json으로 저장

In [25]:
# chapters_list.json으로 저장
with open('./data/json/chapters_list.json', 'w', encoding='utf-8') as f:
    json.dump(chapters_unique2, f, ensure_ascii=False)
chapters_unique2[0]

{'chapter_id': 1,
 'chapter_name': '몇일까요 (1)',
 'school_level': '초등',
 'grade_level': '초1',
 'semester': '1학기',
 'chapter_main': '',
 'chapter_sub': '9까지의 수'}

### ㅇ insert.sql로 저장

In [26]:
# insert_chapters.sql

file_path = "../api/sql/insert_chapters.sql"

with open(file_path, "w", encoding='utf-8') as file:
    for chapter in chapters_unique2 :
        chapter_id = chapter['chapter_id']
        chapter_name = chapter['chapter_name']
        school_level = chapter['school_level']
        grade_level = chapter['grade_level']
        semester = chapter['semester']
        chapter_main = chapter['chapter_main']
        chapter_sub = chapter['chapter_sub']
        sql = f"INSERT INTO chapters VALUES ({chapter_id}, '{chapter_name}', '{school_level}', '{grade_level}', '{semester}', '{chapter_main}', '{chapter_sub}');"
        file.write(sql + "\n")

print("SQL 문장들 insert_chapters.sql 파일로 저장 성공")

SQL 문장들 insert_chapters.sql 파일로 저장 성공


In [27]:
# insert_concepts.sql

file_path = "../api/sql/insert_concepts.sql"

with open(file_path, "w", encoding='utf-8') as file:
    for concept in concepts_sorted :
        concept_id = concept['id']
        concept_name = concept['name']
        description = concept['description']
        concept_chapter_id = concept['concept_chapter_id']
        achievement_id = concept['achievement_id']
        achievement_name = concept['achievement_name']
        skill_id = concept['skill_id']
        sql = f"INSERT INTO concepts VALUES ({concept_id}, '{concept_name}', '{description}', {concept_chapter_id}, {achievement_id}, '{achievement_name}', {skill_id});"
        file.write(sql + "\n")

print("SQL 문장들 insert_concepts.sql 파일로 저장 성공")

SQL 문장들 insert_concepts.sql 파일로 저장 성공


### ㅇ dic으로 저장

In [28]:
# dic으로 저장
concepts_dict = {}
for concept in concepts_sorted :
    concepts_dict[concept.pop('id')] = concept
    
with open('./data/json/concepts_dict.json', 'w', encoding='utf-8') as f:
    json.dump(concepts_dict, f, ensure_ascii=False)

concepts_dict[5814]

{'name': '1부터 5까지의 수',
 'description': '1부터 5까지의 수를 읽고 셀 수 있다.',
 'concept_chapter_id': 1,
 'achievement_id': 1,
 'achievement_name': '50까지의수 개념을이해 하고, 수를 세고 읽고 쓸 수 있다.',
 'skill_id': 8}